In [3]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import os
import tqdm
import rasterio.features
from shapely.geometry import shape
import pandas as pd



shp_file = '../1.Initilization/intermediate/unique_polygons_shapefile.shp'
gdf_mask = gpd.read_file(shp_file)
gdf_mask = gdf_mask.to_crs("EPSG:4326")
#loop through the raster files
raster_file = r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\0.external_data\cropland_rangeland_mask\asap_mask_rangeland_v02.tif'

# Read the raster file (.tif)
src = rasterio.open(raster_file)
# Get the geometry of the shapefile as GeoJSON
mask_geom = gdf_mask.geometry


def raster_to_polygons(raster_array, transform, nodata=None):
    polygons = []
    for vec, val in rasterio.features.shapes(raster_array, mask=raster_array != nodata, transform=transform):
        if val != nodata:
            polygons.append({"geometry": shape(vec), "value": val})
    return polygons

# Mask the raster using the geometry
out_image, out_transform = mask(src, mask_geom, crop=True, filled=False)
out_meta = src.meta.copy()
out_meta.update({"driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform})
polygons = raster_to_polygons(out_image[0], out_transform)
# Create a GeoDataFrame from the polygons
gdf = gpd.GeoDataFrame(polygons, crs=out_meta["crs"])

# Set a new index if desired
result = gdf.reset_index().rename(columns={0: 'value'})


# Perform a spatial join between the points and polygons
spatial_join_gdf = gpd.sjoin(result, gdf_mask, how='inner', op='within')

# Group by the 'polygon_id' column and calculate the mean of the 'value' column
aggregated_gdf = spatial_join_gdf.groupby('FID').agg({'value': 'mean'}).reset_index()

# join aggregated_gdf with gdf_mask on FID
aggregated_gdf = gdf_mask.merge(aggregated_gdf, on='FID', how='left')


rename_dict = {'FID': 'polygon_id', 'value': 'rangeland_mask'}
aggregated_gdf = aggregated_gdf.rename(columns=rename_dict)

# read geojson
geojson_file = '../2.merge_data/intermediate/output7.geojson'


# read geojson
gdf = gpd.read_file(geojson_file)

# drop
gdf = gdf.drop(columns=['index_right','polygon_id'])

# spatial join
gdf_2 = gpd.sjoin(gdf, aggregated_gdf, how='left', op='within')
# drop
gdf_2 = gdf_2.drop(columns=['index_right','polygon_id'])


c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [ ]:
# output
gdf_2.to_file('../2.merge_data/intermediate/output8.geojson', driver='GeoJSON')